## UMAP reproducibility benchmark (runtime & trustworthiness)

In [1]:
from umap import UMAP as umap_learn
from cuml.manifold import UMAP as umap_cuml
from cuml.metrics import trustworthiness

from sklearn.datasets import make_blobs
import time
import numpy as np

In [2]:
def benchmark(args):
    # Generate dataset
    X, y = make_blobs(n_samples=args['n_samples'], n_features=args['n_features'],
                      centers=args['centers'])
    
    # Warm up (used to prevent statistical anomalies in time measurement due to first time initialization)
    model = umap_learn(n_neighbors=args['n_neighbors'], n_epochs=args['n_epochs'],
                 random_state=args['random_state'])
    model.fit_transform(X)
    
    durations = []
    trust_scores = []
    for i in range(args['n_iter']):
        # Instantiate UMAP-learn model
        model = umap_learn(n_neighbors=args['n_neighbors'], n_epochs=args['n_epochs'],
                     random_state=args['random_state'])
        
        # Perform transformation and measure time
        start = time.time()
        transformed = model.fit_transform(X)
        durations.append(time.time()-start)
        
        # Compute trustworthiness score
        trust_scores.append(trustworthiness(X, transformed, n_neighbors=args['n_neighbors']))
    durations = np.array(durations)
    trust_scores = np.array(trust_scores)
    
    # Compute runtime average and variance as well as trustworthiness score average
    umap_learn_scores = durations.mean(), durations.var(), trust_scores.mean()
    
    
    # Warm up (used to prevent statistical anomalies in time measurement due to first time initialization)
    model = umap_cuml(n_neighbors=args['n_neighbors'], n_epochs=args['n_epochs'],
                 random_state=args['random_state'])
    model.fit_transform(X)
    
    durations = []
    trust_scores = []
    for i in range(args['n_iter']):
        # Instantiate cuML's UMAP model
        model = umap_cuml(n_neighbors=args['n_neighbors'], n_epochs=args['n_epochs'],
                     random_state=args['random_state'])
        
        # Perform transformation and measure time
        start = time.time()
        transformed = model.fit_transform(X)
        durations.append(time.time()-start)
        
        # Compute trustworthiness score
        trust_scores.append(trustworthiness(X, transformed, n_neighbors=args['n_neighbors']))
        
    durations = np.array(durations)
    trust_scores = np.array(trust_scores)
    
    # Compute runtime average and variance as well as trustworthiness score average
    umap_cuml_scores = durations.mean(), durations.var(), trust_scores.mean()
    
    duration_mean, duration_var, trust = umap_learn_scores
    print("\tUMAP-LEARN: duration avg - var: {:.2f} - {:.2f}, tustworthiness: {:.2f}".format(duration_mean, duration_var, trust))
    duration_mean, duration_var, trust = umap_cuml_scores
    print("\tCUML UMAP: duration avg - var: {:.2f} - {:.2f}, tustworthiness: {:.2f}".format(duration_mean, duration_var, trust))
    
    return umap_learn_scores, umap_cuml_scores

In [3]:
import warnings
warnings.filterwarnings('ignore')

args = {'n_samples':20000, 'n_features':100, 'centers':500,
        'n_neighbors':15, 'n_epochs':500, 'n_iter': 5}

print("Without random state :")
args['random_state'] = None
umap_learn_scores, umap_cuml_scores = benchmark(args)

print("\nWith random state :")
args['random_state'] = 42
umap_learn_scores, umap_cuml_scores = benchmark(args)

Without random state :
	UMAP-LEARN: duration avg - var: 53.22 - 0.04, tustworthiness: 1.00
	CUML UMAP: duration avg - var: 0.28 - 0.00, tustworthiness: 1.00

With random state :
	UMAP-LEARN: duration avg - var: 53.05 - 0.04, tustworthiness: 1.00
	CUML UMAP: duration avg - var: 0.38 - 0.00, tustworthiness: 1.00
